In [47]:
import pandas as pd

# Ler o arquivo JSON
df = pd.read_json('quinto_ap.json')
df_cob = pd.read_json('quinto_cob.json')


In [48]:
df_cob

,_id,_score,_source,fields
0,894369944,2.51,"{'id': 894369944, 'type': 'Apartamento', 'area...",NaN
1,894467378,2.51,"{'id': 894467378, 'type': 'Apartamento', 'area...",{'listingTags': ['NEW_AD']}
2,894134883,2.51,"{'id': 894134883, 'type': 'Apartamento', 'area...","{'listingTags': ['RENT_PRICE_DECREASED', 'RENT..."
3,894148058,2.51,"{'id': 894148058, 'type': 'Apartamento', 'area...",NaN
4,894542061,2.51,"{'id': 894542061, 'type': 'Apartamento', 'area...",{'listingTags': ['NEW_AD']}
...,...,...,...,...
93,894413032,2.51,"{'id': 894413032, 'type': 'Apartamento', 'area...",{'listingTags': ['RENT_PRICE_DECREASED']}
94,894410145,2.51,"{'id': 894410145, 'type': 'Apartamento', 'area...",NaN
95,894383473,2.51,"{'id': 894383473, 'type': 'Apartamento', 'area...",{'listingTags': ['RENT_PRICE_DECREASED']}
96,894410166,2.51,"{'id': 894410166, 'type': 'Apartamento', 'area...",NaN


In [49]:
def process_dataframe(df):
    # Remover a coluna 'fields'
    del df['fields']
    
    # Expandir o JSON na coluna '_source'
    df_expanded = pd.json_normalize(df['_source'])
    
    # Filtrar linhas onde 'forRent' é True
    df = df_expanded[df_expanded['forRent'] == True]
    
    # Selecionar colunas específicas
    selected_columns = ['id', 'type', 'area', 'bathrooms', 'bedrooms', 'parkingSpaces', 'rent', 'iptuPlusCondominium', 'address', 'city', 'regionName']
    df_result = df[selected_columns]
    
    # Remover duplicatas com base na coluna 'id'
    df_result = df_result.drop_duplicates('id')
    
    # Remover a coluna 'id'
    df_result.drop(columns=['id'], inplace=True)
    
    # Filtrar linhas onde 'city' é 'Belo Horizonte'
    df_result = df_result[df_result['city'] == 'Belo Horizonte']
    
    # Remover a coluna 'city'
    del df_result['city']
    
    # Filtrar linhas onde 'type' é 'Apartamento'
    df_result = df_result[df_result['type'] == 'Apartamento']
    
    # Remover a coluna 'type'
    del df_result['type']
    
    # Converter a coluna 'parkingSpaces' para int
    df_result['parkingSpaces'] = df_result['parkingSpaces'].astype(int)

    df_result  = df_result.rename(columns={
    'rent': 'price',
    'bedrooms': 'rooms',
    'parkingSpaces': 'parking',
    'bathrooms': 'toilets',
    'area': 'size'
})
    
    return df_result


In [50]:
df = process_dataframe(df)
df_cob = process_dataframe(df_cob)

In [51]:
df_cob['penthouse'] = 1
df['penthouse'] = 0
df_final = pd.concat([df, df_cob], ignore_index=True)
df_final

,size,toilets,rooms,parking,price,iptuPlusCondominium,address,regionName,penthouse
0,64,1,2,1,2300,380,Rua Bernardino Oliveira Pena,São João Batista,0
1,48,1,2,1,1100,367,Rua Santo Antônio,São João Batista,0
2,60,1,2,1,950,251,Rua Princesa Isabel,Serra Verde,0
3,60,1,2,1,1761,346,Rua Maria de Souza Alves,Manacás,0
4,42,1,2,1,2100,380,Rua Juarez,União,0
...,...,...,...,...,...,...,...,...,...
906,210,4,4,4,11630,1034,Rua Castelo de Edimburgo,Castelo,1
907,220,3,5,2,4875,780,Rua Lago do Junco,Jardim Atlântico,1
908,207,4,3,4,12000,550,Rua dos Expedicionários,Santa Amélia,1
909,280,3,4,2,5370,539,Rua Teodomiro Cruz,São Lucas,1


In [52]:
def remove_duplicates_with_priority(df):
    """
    Remove duplicatas de um DataFrame com base em colunas específicas,
    mantendo as linhas onde a coluna 'penthouse' é 1.
    
    """
    # Ordenar para que as linhas com 'penthouse' == 1 venham primeiro
    df = df.sort_values(by='penthouse', ascending=False)

    # Remover duplicatas mantendo a primeira ocorrência (onde 'penthouse' == 1)
    df = df.drop_duplicates(subset=['price', 'regionName', 'address', 'rooms', 'parking', 'toilets', 'size', 'iptuPlusCondominium'])

    return df

In [53]:
df_result = remove_duplicates_with_priority(df_final)

In [54]:
df_result.to_json('dados/quinto.json', orient='records', force_ascii=False, indent=4)